## Import packages

In [382]:
import pandas as pd
import numpy as np
import sys 
from tqdm import tqdm
tqdm.pandas() 

sys.path.append("..")

from src.support_cleaning import normalize, fill_categories_forward_backward_massive, find_similar_values

## Data import

In [ ]:
brasil_public = pd.read_parquet("../data/concatenated_data.parquet")
brasil_public.info()
brasil_public.head()

## Data cleaning

First problems to correct as identified in exploration:
- Data types for numerical features valor_previsto_atualizado, valor_lancado, valor_realizado, percentual_realizado and datetime data_lancamento
- Missing values in nome_orgao_superior, as well as other columns, to be inferred from other columns and rows
- Duplicated values

### 2.1 Correcting data types

In [384]:
data_types_dict = {
    "codigo_orgao_superior": object,
    "codigo_orgao": object,  
    "codigo_unidade_gestora": object,      
    "valor_previsto_atualizado": float,
    "valor_lancado": float,  
    "valor_realizado": float,      
    "percentual_realizado": float,
    "data_lancamento": "datetime64[ns]"
}

#### 2.1.1 Replacing floating commas by floating point

In [385]:
for column, data_type in data_types_dict.items():
    if data_type == float:
        brasil_public[column] = brasil_public[column].str.replace(",",".")

#### 2.1.2 Correcting data types

In [ ]:
brasil_public = brasil_public.astype(data_types_dict)
brasil_public.info()

In [ ]:
brasil_public.describe().T.assign(missing_values= lambda x: brasil_public.shape[0] - x["count"]).T

Now that the numerical values have been corrected, the revenue value columns can be explored. Odd things immediately struck when seeing the negative numbers as minimum values. Theoretically, revenue should always be positive and expenditures should appear in different reports, so one possible explanation for this could be that certain corrections to the same category are made.

In [ ]:
brasil_public.describe(include=['O']).T.assign(missing_values= lambda x: brasil_public.shape[0] - x["count"])

### 2.1 Normalization of categories

#### 2.1.1 Addition of new category 'intra_orcamentaria'

In [ ]:
brasil_public[['categoria_economica','intra_orcamentaria']] = brasil_public['categoria_economica'].str.split(" - ",expand=True)
brasil_public['intra_orcamentaria'].replace("intra-orçamentárias", "Yes",inplace=True)
brasil_public['intra_orcamentaria'].fillna("No", inplace=True)

In [ ]:
brasil_public['intra_orcamentaria']

## 2.1 Normalization of categories

#### 2.1.2 Correction of erratum

Find similar values is based on the Levensthein distance, that measures the number of character changes needed inside an element of a pair of strings for the pair to be equal. This can narrow down the exploration of erratum.

In [ ]:
for column in brasil_public.select_dtypes("O").filter(regex='^(?!codigo)').columns.to_list():
    print(f"\n\n\n{column}")
    similar_pairs = find_similar_values(brasil_public[column], distance=2)
    print(similar_pairs)


Some results of visual inspection for erratum were:

- **"Outros receitas de serviços"** vs. **"outras receitas de serviços"**: "Outros" should be "outras."

- **"Receitas correntes - a classificar"** vs. **"receitas correntes a classificar"**: Hyphen inconsistency. Replace by underscore.

- **"Receita_da_industria_de_prod._farmoquimicos"** vs. **"receita_da_industria_de_prod.farmoquimicos"**: Underscore inconsistency. Replace dots by underscores and replace multiple underscores by one.

- **"Remuneração das disponibilidades do tesouro n"** vs. **"remuneração das disponibilidades do tesouro"**: "n" might refer to "nacional."

- **"Receitas_correntes___a_classificar"** vs. **"receitas_correntes_a_classificar"**: Excessive spacing. Replace multiple underscores by one.

- **"Serviço de reparação, manutenção e instalação"** vs. **"serviços de reparação, manutenção e instalação"**: Singular vs. plural inconsistency.

- **"Transferências de outras instituições pública"** vs. **"transferência de outras instituições públicas"**: Singular vs. plural inconsistency.

- **"Cont.pis/pasep-ñ opt.simp.nac-multas div.at."** vs. **"cont.pis/pasep-opt.simp.nac-multas div.at."**: Replace "ñ " by "".

- **"Contribuição para o pis/pasep-juros div.ativa"** vs. **"contribuição para o pis/pasep-juros-div.ativa"**: Replace hyphen and space with an underscore.



In [392]:
brasil_public.iloc[:,:10] = brasil_public.iloc[:,:10].map(lambda x: normalize(str(x)) if not pd.isna(x) else x)

Erratum have been already corrected in this last output, as the correction has been made in the form of modification to the support function ``normalize()``. Other specific corrections are made below:

In [393]:
replacements = (
    ("outros","outras"),
    ("tesouro_n","tesouro"),
    ("servicos","servico"),
    ("desenv","desenvol"),
)
for a, b in replacements:
    brasil_public["detalhamento"] = brasil_public["detalhamento"].str.replace(a,b)

After these corrections, categories should have rather consistent namings, altough some possibilities for inconsistency may still exist within the 'detalhamento' and 'nome_unidade_gestora' features. Due to its high cardinality and specificity of its used acronyms, complete assurance of correction is difficult and bears a low ROI on effort for the analysis, so it is considered complete.

### 2.2 Missing values

Columns in the dataset have each the following number of missing values:

In [ ]:
brasil_public.isna().sum()

In principle, all features with name and code (i.e. 'codigo_orgao_superior' and 'nome_orgao_superior') should have a one-to-one relationship with each other. If there are cases where not both corresponding values are missing, then it should allow for a correct filling of values based off its equivalent.

The same can be done with the seeming hierarchical features of categoria_economica, origem_receita and especie_receita, albeit more unidirectionally than with the previously mentioned pairs of features.

To end the filling of missing values topic, it is possible that monetary features 'valor_previsto_atualizado', 'valor_realizado' and 'percentual_realizado' also offer the possibility of filling in the missing values according to the formula that relates them.

Finally, on another note, the remaining missing will be evaluated to either:
- Drop the rows, in case they hold a percentage of missing that does not offer information to the analysis.
- Impute the value, very improbable and left to be evaluated at the time of the analysis. As imputing values by approximating them to a statistic or other techniques is delicate and requires of careful measure.

#### 2.2.1 Filling missing values

##### 2.2.1.1 Codigo orgao superior & nome_orgao_superior

As codigo_orgao_superior and nome_orgao_superior should bear a one to one relationship, filling the gaps with one another can be a good option. To do that, a mapping will be used, that takes a generated equivalence dictionary from the columns like so:

In [ ]:
codigo_nome_orgao_superior = brasil_public[["codigo_orgao_superior","nome_orgao_superior"]].value_counts().index.to_list()
codigo_nome_orgao_superior_dict = {codigo: nome for codigo, nome  in codigo_nome_orgao_superior}
codigo_nome_orgao_superior_dict_reverse = {column2: column1 for column1, column2 in codigo_nome_orgao_superior_dict.items()}
codigo_nome_orgao_superior_dict

Applying the mapping through a fill na is done as per the following:

In [396]:
brasil_public['nome_orgao_superior_filled'] = brasil_public['nome_orgao_superior'].fillna(brasil_public['codigo_orgao_superior'].map(codigo_nome_orgao_superior_dict))
brasil_public['codigo_orgao_superior_filled'] = brasil_public['codigo_orgao_superior'].fillna(brasil_public['nome_orgao_superior'].map(codigo_nome_orgao_superior_dict_reverse))

Now, checking that the filling works correctly:

In [ ]:
# check filled vs original missing
display(brasil_public[["codigo_orgao_superior","nome_orgao_superior",'codigo_orgao_superior_filled', 'nome_orgao_superior_filled']].isna().sum())

missing_filter_codigo = brasil_public["codigo_orgao_superior"].isna() 
missing_filter_nome=  brasil_public["nome_orgao_superior"].isna()

# printing missing code
display(brasil_public.loc[missing_filter_codigo,["codigo_orgao_superior","nome_orgao_superior",'codigo_orgao_superior_filled', 'nome_orgao_superior_filled']].head())

# printing missing name
brasil_public.loc[missing_filter_nome,["codigo_orgao_superior","nome_orgao_superior",'codigo_orgao_superior_filled', 'nome_orgao_superior_filled']].head()

The check confirms that columns where 'nome_orgao_superior' was missing now have the correct name. None values are essentially the same as NaN, so they are converted as such.

Now the rest of pairs can be filled with the same technique.

##### 2.2.1.2 Codigo_orgao & nome_orgao

To make the code cleaner, the creation of the equivalences dictionary and the application of the filling function have been included in a higher support function.

In [ ]:
codigo_nome_orgao = brasil_public[["codigo_orgao","nome_orgao"]].value_counts().index.to_list()
codigo_nome_orgao_dict = {codigo: nome for codigo, nome  in codigo_nome_orgao}
codigo_nome_orgao_dict_reverse = {column2: column1 for column1, column2 in codigo_nome_orgao_dict.items()}
codigo_nome_orgao_dict

In [399]:
brasil_public['nome_orgao_filled'] = brasil_public['nome_orgao'].fillna(brasil_public['codigo_orgao'].map(codigo_nome_orgao_dict))
brasil_public['codigo_orgao_filled'] = brasil_public['codigo_orgao'].fillna(brasil_public['nome_orgao'].map(codigo_nome_orgao_dict_reverse))

In [ ]:
# check filled vs original missing
display(brasil_public[["codigo_orgao","nome_orgao",'codigo_orgao_filled', 'nome_orgao_filled']].isna().sum())

missing_filter_codigo = brasil_public["codigo_orgao"].isna() 
missing_filter_nome=  brasil_public["nome_orgao"].isna()

# printing missing code
display(brasil_public.loc[missing_filter_codigo,["codigo_orgao","nome_orgao",'codigo_orgao_filled', 'nome_orgao_filled']].head())

# printing missing name
brasil_public.loc[missing_filter_nome,["codigo_orgao","nome_orgao",'codigo_orgao_filled', 'nome_orgao_filled']].head()

##### 2.2.1.3 codigo_unidade_gestora & nome_unidade_gestora

In [ ]:
codigo_nome_unidade_gestora = brasil_public[["codigo_unidade_gestora","nome_unidade_gestora"]].value_counts().index.to_list()
codigo_nome_unidade_gestora_dict = {codigo: nome for codigo, nome  in codigo_nome_unidade_gestora}
codigo_nome_unidade_gestora_dict_reverse = {column2: column1 for column1, column2 in codigo_nome_unidade_gestora_dict.items()}
codigo_nome_unidade_gestora_dict

In [402]:
brasil_public['nome_unidade_gestora_filled'] = brasil_public['nome_unidade_gestora'].fillna(brasil_public['codigo_unidade_gestora'].map(codigo_nome_unidade_gestora_dict))
brasil_public['codigo_unidade_gestora_filled'] = brasil_public['codigo_unidade_gestora'].fillna(brasil_public['nome_unidade_gestora'].map(codigo_nome_unidade_gestora_dict_reverse))

In [ ]:
# check filled vs original missing
display(brasil_public[["codigo_unidade_gestora","nome_unidade_gestora",'codigo_unidade_gestora_filled', 'nome_unidade_gestora_filled']].isna().sum())

missing_filter_codigo = brasil_public["codigo_unidade_gestora"].isna() 
missing_filter_nome=  brasil_public["nome_unidade_gestora"].isna()

# printing missing code
display(brasil_public.loc[missing_filter_codigo,["codigo_unidade_gestora","nome_unidade_gestora",'codigo_unidade_gestora_filled', 'nome_unidade_gestora_filled']].head())

# printing missing name
brasil_public.loc[missing_filter_nome,["codigo_unidade_gestora","nome_unidade_gestora",'codigo_unidade_gestora_filled', 'nome_unidade_gestora_filled']].head()

##### 2.2.1.4 categoria_economica & origem_receita & especie_receita 

These revenues groups should share a non-shared hierarchy, meaning that a origem_receita only has a categoria_economica. If that is the case, a unidirectional filling from origem_receita to categoria_economica could be done. That could also be the case for especie_receita.

Let's check that by visually inspecting the value counts, first ordered by origem_receita with respect to categoria_economica. 

In [ ]:
pd.set_option("display.max_rows",85)
brasil_public[['categoria_economica', 'origem_receita']].value_counts().reset_index().sort_values(by="origem_receita")

As suspected, each categoria_economica bears a non-shared hierarchy with respect to origem_receita, which then means that the previous method for filling NaNs can again be used. Still, there would be just one modification to make for this to work and that is to create a separate column for receitas 'intra-orcamentárias' [Yes/no] to remove special cases. 

In [ ]:
brasil_public[['categoria_economica']]

In [ ]:
brasil_public[['categoria_economica', 'origem_receita']].value_counts().reset_index().sort_values(by="origem_receita")

In [407]:
categoria_economica_receita = (brasil_public[["origem_receita","categoria_economica"]]
                                            .value_counts()
                                            .index.to_list())

categoria_economica_receita_dict = {origem: categoria for origem, categoria  in categoria_economica_receita}

brasil_public['categoria_economica_filled'] = (brasil_public['categoria_economica']
                                                .fillna(brasil_public['origem_receita']
                                                .map(categoria_economica_receita_dict)))

In [ ]:
brasil_public[["origem_receita","categoria_economica"]].value_counts()

In [ ]:
categoria_economica_receita_dict

In [ ]:
# check filled vs original missing
display(brasil_public[["categoria_economica", "categoria_economica_filled"]].isna().sum())

missing_filter_categoria = brasil_public["categoria_economica"].isna() 
missing_filter_origem_receita=  brasil_public["origem_receita"].isna()

# printing missing code
display(brasil_public.loc[missing_filter_categoria,["categoria_economica","categoria_economica_filled"]].head())


Again, the same technique can be repeated for especie receita and origem receita if the conditions apply. Let's check their unique combinations:

In [ ]:
display(brasil_public[['origem_receita', 'especie_receita']].value_counts().reset_index().sort_values(by="especie_receita"))
pd.set_option("display.max_rows",20)

It seems that the equivalence still applies, but only for those especie_receitas that do not start with "Transferências". The rule must be that, for origem_receita to be filled, especie_receita must only correspond with one origem_receita. Let's then just take the especie_receita that appear only once in the value_counts().

In [ ]:
origem_especie_receita = brasil_public[['origem_receita', 'especie_receita']].value_counts().reset_index()
print(f"Full unique combinations: {origem_especie_receita.shape[0]}")
origem_especie_receita.drop_duplicates(subset="especie_receita",keep=False,inplace=True)
print(f"Unique combinations of one especie_receita: {origem_especie_receita.shape[0]}")

origem_especie_receita = origem_especie_receita[["origem_receita","especie_receita"]].values

origem_especie_receita_dict = {detalhamento: especie for especie, detalhamento  in origem_especie_receita}

brasil_public['origem_receita_filled'] = (brasil_public['origem_receita']
                                                .fillna(brasil_public['especie_receita']
                                                .map(origem_especie_receita_dict)))

In [ ]:
# check filled vs original missing
display(brasil_public[["origem_receita", "origem_receita_filled"]].isna().sum())

missing_filter_origem_receita = brasil_public["origem_receita"].isna()

# printing missing code
display(brasil_public.loc[missing_filter_origem_receita,["origem_receita","origem_receita_filled","especie_receita"]].head())


##### 2.2.1.5 especie_receita & detalhamento 

To fill the especie_receita, things get more difficult as there are many detalhamentos and that means that they might be shared among especie_receitas. Thus, a method to find the coincidences could be to find the value_counts of both features and keep the detalhamentos that appear extrictly once.

In [ ]:
especie_receita_detalhamentos = brasil_public[['especie_receita', 'detalhamento']].value_counts().reset_index()
print(f"Full unique combinations: {especie_receita_detalhamentos.shape[0]}")
especie_receita_detalhamentos.drop_duplicates(subset="detalhamento",keep=False,inplace=True)
print(f"Unique combinations of one detalhamento: {especie_receita_detalhamentos.shape[0]}")

It seems that there are quite enough detalhamentos that are not shared among especie_receitas, which makes room for filling especie_receitas with it.

In [ ]:
especie_receita_detalhamentos = especie_receita_detalhamentos[["especie_receita","detalhamento"]].values

especie_receita_detalhamentos_dict = {detalhamento: especie for especie, detalhamento  in especie_receita_detalhamentos}

brasil_public['especie_receita_filled'] = (brasil_public['especie_receita']
                                                .fillna(brasil_public['detalhamento']
                                                .map(especie_receita_detalhamentos_dict)))

# check filled vs original missing
display(brasil_public[["especie_receita", "especie_receita_filled"]].isna().sum())

missing_filter_origem_receita = brasil_public["especie_receita"].isna()

# printing missing code
display(brasil_public.loc[missing_filter_origem_receita,["especie_receita","especie_receita_filled","detalhamento"]].head())

##### 2.2.1.6 Filling all categorical values at once

When filling gaps from one column to another, as they fill in each new pair of columns, new values appear that would have filled and could fills missings for already processed pairs. Thus, a backward-forward functional approach can be handy for to this process to be re-applied to fill-in the now fillable gaps. 

The function for that process that has been coded in the ``support_cleaning.py`` file and its called ``fill_categories_forward_backward_massive()``. One thing to note in this funciton is that equivalences are pairwise for neighboring columns, and there might be some room for gap filling between non-adjacent columns, but that is left for future steps, as reduction of missings via this mecanism is already satisfactory.

Having checked for each column that the missing filling step is correct, the first 10 columns can directly be overwritten by its '_filled' counterparts.

In [ ]:
brasil_public.iloc[:,0:10:2] = fill_categories_forward_backward_massive(brasil_public.iloc[:,0:10:2])

In [ ]:
brasil_public.iloc[:,:10] = fill_categories_forward_backward_massive(brasil_public.iloc[:,:10])

In [ ]:
brasil_public.isna().sum()

Below, a comparison between the gap-filling operation carried out "manually" and the new function. There are cases where the manual holds less missing values. The reason for this is that during the visual inspection of categories, some equivalences were incorrectly identified as unique, and thus some values were incorrectly filled.

In [ ]:
for column in brasil_public.iloc[:,:9].columns:
    manual = brasil_public[f"{column}_filled"].isna().sum()
    forward_backward = brasil_public[column].isna().sum()
    print(f"{column.capitalize()}: Manual {manual} VS forward-backward {forward_backward}")

Finally, "_filled" columns are dropped.

In [ ]:
brasil_public = brasil_public.iloc[:,:-9]
brasil_public

##### 2.2.1.7 ano_exercicio

In [421]:
brasil_public.loc[brasil_public["ano_exercicio"].isna(),"ano_exercicio"] = (brasil_public
                                                                            .loc[brasil_public["ano_exercicio"].isna(),"fichero"]
                                                                            .str[-4:]).astype(int)

##### 2.2.1.8 valor_previsto_atualizado, valor_lancado, valor_realizado, percentual_realizado

In [ ]:
brasil_public[['valor_previsto_atualizado', 'valor_lancado',
       'valor_realizado', 'percentual_realizado']].describe().T.assign(missing_values = lambda x: brasil_public.shape[0] - x["count"] )

In [ ]:
valor_previsto_atualizado_not_null = ~brasil_public['valor_previsto_atualizado'].isna()
valor_lancado_not_null = ~brasil_public['valor_lancado'].isna()
valor_realizado_not_null = ~brasil_public['valor_realizado'].isna()
percentual_realizado_not_null = ~brasil_public['percentual_realizado'].isna()


brasil_public.loc[valor_previsto_atualizado_not_null,['valor_previsto_atualizado', 'valor_lancado',
       'valor_realizado', 'percentual_realizado']].query("valor_previsto_atualizado != 0")

There are null values for percentual_realizado that could be calculated from valor previsto. Also, which is more strange, there are values worth zero for this same feature that should worth something, based on the values present for valor_previsto_atualizado and valor_realizado.

In [ ]:
percentual_realizado_not_null_or_zero = brasil_public['percentual_realizado'].isna() | brasil_public['percentual_realizado'] == 0
brasil_public.loc[percentual_realizado_not_null_or_zero,['valor_previsto_atualizado', 'valor_lancado',
       'valor_realizado', 'percentual_realizado']]

In [ ]:
percentual_realizado_null_or_zero = (brasil_public['percentual_realizado'].isna()) | (brasil_public['percentual_realizado'] == 0)
valor_previsto_atualizado_not_null_nor_zero = (~brasil_public['valor_previsto_atualizado'].isna()) & brasil_public['valor_previsto_atualizado'] != 0
valor_realizado_not_null= ~brasil_public['valor_realizado'].isna()

filters = percentual_realizado_null_or_zero & valor_previsto_atualizado_not_null_nor_zero & valor_realizado_not_null

brasil_public.loc[filters,['valor_previsto_atualizado', 'valor_lancado',
       'valor_realizado', 'percentual_realizado']]

In [426]:
percentual_realizado_null_or_zero = (brasil_public['percentual_realizado'].isna()) | (brasil_public['percentual_realizado'] == 0)
valor_previsto_atualizado_not_null_nor_zero = (~brasil_public['valor_previsto_atualizado'].isna()) & (brasil_public['valor_previsto_atualizado'] != 0)
valor_realizado_not_null= ~brasil_public['valor_realizado'].isna()

filters = percentual_realizado_null_or_zero & valor_previsto_atualizado_not_null_nor_zero & valor_realizado_not_null

brasil_public.loc[filters,'percentual_realizado'] = ((brasil_public.loc[filters,'valor_realizado'] / 
                                                    brasil_public.loc[filters,'valor_previsto_atualizado']) * 100)

In [427]:
valor_realizado_null_or_zero = (brasil_public['valor_realizado'].isna()) | (brasil_public['valor_realizado'] == 0)

percentual_realizado_not_null_or_zero = (~brasil_public['percentual_realizado'].isna()) & (brasil_public['percentual_realizado'] != 0)
valor_previsto_atualizado_not_null = ~brasil_public['valor_previsto_atualizado'].isna()
 
filters = valor_realizado_null_or_zero & percentual_realizado_not_null_or_zero & valor_previsto_atualizado_not_null 

brasil_public.loc[filters,'valor_realizado'] = ((brasil_public.loc[filters,'percentual_realizado'] / 100) * 
                                                brasil_public.loc[filters,'valor_previsto_atualizado'])

In [428]:
valor_previsto_atualizado_null_or_zero = (brasil_public['valor_previsto_atualizado'].isna()) | (brasil_public['valor_previsto_atualizado'] == 0)

valor_realizado_not_null_nor_zero = ~brasil_public['valor_realizado'].isna()
percentual_realizado_not_null_or_zero = (~brasil_public['percentual_realizado'].isna()) & (brasil_public['percentual_realizado'] != 0)


filters = valor_previsto_atualizado_null_or_zero & valor_realizado_not_null_nor_zero & percentual_realizado_not_null_or_zero 
brasil_public.loc[filters,'valor_previsto_atualizado'] = ((brasil_public.loc[filters,'valor_realizado'] / 
                                                           brasil_public.loc[filters,'percentual_realizado']) * 100)


In [ ]:
brasil_public[['valor_previsto_atualizado', 'valor_lancado',
       'valor_realizado', 'percentual_realizado']].describe().T.assign(missing_values = lambda x: brasil_public.shape[0] - x["count"] )

### 2.3 Duplicates

In [ ]:
brasil_public.duplicated().sum()

In [431]:
brasil_public.drop_duplicates(inplace=True)

In [432]:
brasil_public.reset_index(drop=True).to_parquet("../data/cleaned_data.parquet")

In [ ]:
brasil_public.info()

In [ ]:
brasil_public.info()